In [1]:
# Autoreload
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

import torch

import datasets
# Don't show progress datasets bars
datasets.disable_progress_bar()

import sys
sys.path.insert(0, str(Path.cwd().parent.resolve()))
from dataset import get_dataset_trlx
# from model import get_model
from reward_model import get_template, get_reward_model, create_reward_fn_trlx
from utils import get_tokenizer

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:12                                                                                   │
│                                                                                                  │
│    9 import sys                                                                                  │
│   10 sys.path.insert(0, str(Path.cwd().parent.resolve()))                                        │
│   11 from dataset import get_dataset_trlx                                                        │
│ ❱ 12 from model import get_model                                                                 │
│   13 from reward_model import get_template, get_reward_model, create_reward_fn_trlx              │
│   14 from utils import get_tokenizer                                                             │
│   15                                                                                             │
│                                                                                                  │
│ /fsx/home-augustas/mlmi-thesis/src/ppo/model.py:3 in <module>                                    │
│                                                                                                  │
│    1 from peft import LoraConfig                                                                 │
│    2                                                                                             │
│ ❱  3 from trl import AutoModelForCausalLMWithValueHead                                           │
│    4                                                                                             │
│    5 from utils import get_model_loading_kwargs                                                  │
│    6                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ModuleNotFoundError: No module named 'trl'

In [3]:
device = torch.device(
    f"cuda:{torch.cuda.current_device()}" if torch.cuda.is_available() else "cpu"
)
device

device(type='cuda', index=0)

In [4]:
tokenizer_name = "gpt2-xl"

# dataset_name = "AugustasM/burns-datasets-VINC-ppo-training-v4"
# reward_model_output_path = "/fsx/home-augustas/logs/unifiedqa-v2-t5-3b-1363200_custom_data_v4_all_20230629_120158_21789"

dataset_name = "AugustasM/burns-datasets-VINC-imdb-ppo-training-v2"
# reward_model_output_path = "/fsx/home-augustas/logs/unifiedqa-v2-t5-3b-1363200_custom_data_imdb_v2_first_20230707_170052_28991"
reward_model_output_path = "/fsx/home-augustas/logs/UQA-3b-custom_data_imdb_v2_final_20230717_200713_36998"

## Dataset

In [5]:
dataset = get_dataset_trlx(dataset_name, eval_dataset_size=280)

prompts = dataset["train"].to_list()
eval_prompts = dataset["eval"].to_list()

len(prompts), len(eval_prompts)

Loading dataset...



╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 dataset = get_dataset_trlx(dataset_name, eval_dataset_size=280)                              │
│   2                                                                                              │
│   3 prompts = dataset["train"].to_list()                                                         │
│   4 eval_prompts = dataset["eval"].to_list()                                                     │
│                                                                                                  │
│ /fsx/home-augustas/mlmi-thesis/src/ppo/dataset.py:112 in get_dataset_trlx                        │
│                                                                                                  │
│   109 def get_dataset_trlx(dataset_name, eval_dataset_size, train_dataset_size=8192, subsets_t   │
│   110 │   print("Loading dataset...\n")                                                          │
│   111 │                                                                                          │
│ ❱ 112 │   dataset = load_dataset(dataset_name, split="train")                                    │
│   113 │                                                                                          │
│   114 │   # Delete certain subsets if desired                                                    │
│   115 │   if subsets_to_delete is not None:                                                      │
│                                                                                                  │
│ /fsx/home-augustas/anaconda3/envs/trlx/lib/python3.10/site-packages/datasets/load.py:1773 in     │
│ load_dataset                                                                                     │
│                                                                                                  │
│   1770 │   )                                                                                     │
│   1771 │                                                                                         │
│   1772 │   # Create a dataset builder                                                            │
│ ❱ 1773 │   builder_instance = load_dataset_builder(                                              │
│   1774 │   │   path=path,                                                                        │
│   1775 │   │   name=name,                                                                        │
│   1776 │   │   data_dir=data_dir,                                                                │
│                                                                                                  │
│ /fsx/home-augustas/anaconda3/envs/trlx/lib/python3.10/site-packages/datasets/load.py:1502 in     │
│ load_dataset_builder                                                                             │
│                                                                                                  │
│   1499 │   if use_auth_token is not None:                                                        │
│   1500 │   │   download_config = download_config.copy() if download_config else DownloadConfig(  │
│   1501 │   │   download_config.use_auth_token = use_auth_token                                   │
│ ❱ 1502 │   dataset_module = dataset_module_factory(                                              │
│   1503 │   │   path,                                                                             │
│   1504 │   │   revision=revision,                                                                │
│   1505 │   │   download_config=download_config,                                                  │
│                                                            

In [6]:
# Dataset templates
dataset_template_path = "AugustasM/burns-datasets-VINC"
template = get_template(dataset_template_path)

Num templates: 1
Is the given text truthful


## Reward model

In [7]:
# # Get the reward model
# reward_model, reward_model_name = get_reward_model(
#     reward_model_output_path, device,
#     supervised=True,
# )
# reward_model_tokenizer = get_tokenizer(reward_model_name)

The current device is cuda:1.

Loading reward model from allenai/unifiedqa-v2-t5-3b-1363200.
is_bf16_possible=True


Loaded reward model with 2,851,598,336 parameters.
Number of trainable params 0 parameters.
Reward model dtype: torch.bfloat16

layer=13
Loading the probe from /fsx/home-augustas/VINC-logs/allenai/unifiedqa-v2-t5-3b-1363200/AugustasM/burns-datasets-VINC-imdb-v2/silly-leavitt/lr_models/layer_13.pt
Loaded the probe.

Loading tokenizer allenai/unifiedqa-v2-t5-3b-1363200...
Falling back to slow tokenizer; fast one failed: 'No such file or directory (os error 2)'
Loaded tokenizer.



In [8]:
rm_batch_size = 128

In [9]:
prompts = dataset["train"]["prompt"][:rm_batch_size]
responses = dataset["train"]["original_output"][:rm_batch_size]
texts = [q + r for q, r in zip(prompts, responses)]

len(texts), texts[0]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 prompts = dataset["train"]["prompt"][:rm_batch_size]                                         │
│   2 responses = dataset["train"]["original_output"][:rm_batch_size]                              │
│   3 texts = [q + r for q, r in zip(prompts, responses)]                                          │
│   4                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'dataset' is not defined

In [10]:
# # Create reward function
# reward_fn = create_reward_fn_trlx(
#     reward_model=reward_model,
#     reward_model_tokenizer=reward_model_tokenizer,
#     rm_batch_size=rm_batch_size,
#     template=template,
#     device=device,
#     delta_reward=False,
# )

In [11]:
# rewards = reward_fn(texts, prompts, responses)

# print(rewards.shape)
# rewards

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 rewards = reward_fn(texts, prompts, responses)                                               │
│   2                                                                                              │
│   3 print(rewards.shape)                                                                         │
│   4 rewards                                                                                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'texts' is not defined

## Model